In [53]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
iris = load_iris()

In [54]:
data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [55]:
X = data.drop(['target'], axis=1)
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [56]:
y = data['target']
y

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
145    2.0
146    2.0
147    2.0
148    2.0
149    2.0
Name: target, Length: 150, dtype: float64

In [68]:
# dividir el conjunto de datos en dos, uno para entrenamiento y otro para validación
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(112, 4) (38, 4) (112,) (38,)


In [69]:
# Crear un modelo de regresión lineal.
from sklearn.linear_model import LinearRegression

regres = LinearRegression().fit(X_train, y_train)

In [70]:
# Al comparar el rendimiento del modelo en entrenamiento y validación nos encontramos con un problema: 
# el modelo sobreajuste. Algo que se puede ver en la gran diferencia de rendimiento que se observa en los conjuntos de datos de entrenamiento y validación.
print(f'Entrenamiento { regres.score(X_train, y_train) : .3f}')
print(f'Validación { regres.score(X_test, y_test) : .3f}')

Entrenamiento  0.923
Validación  0.949


In [71]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(f_regression, k=2)

best_train = selector.fit_transform(X_train, y_train)
best_test = selector.transform(X_test)

In [72]:
regres = LinearRegression().fit(best_train, y_train)
print(f'Entrenamiento { regres.score(best_train, y_train) : .3f}')
print(f'Validación { regres.score(best_test, y_test) : .3f}')

Entrenamiento  0.917
Validación  0.948


In [73]:
# Cuales son las columnas elegidas
np.where(selector.get_support())

(array([2, 3], dtype=int64),)

In [76]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipe = Pipeline([('kbest', SelectKBest(f_regression)),
                 ('lineal', LinearRegression())])
grid = GridSearchCV(pipe, {'kbest__k': range(1,20)}, cv=3)
grid.fit(X, y)


c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
45 fits failed out of a total of 57.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\pipeline.py", line 348, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:\Users\Lenovo\ana

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kbest',
                                        SelectKBest(score_func=<function f_regression at 0x00000130217DD430>)),
                                       ('lineal', LinearRegression())]),
             param_grid={'kbest__k': range(1, 20)})

In [77]:
# Mediante GridSearchCV se prueba diferentes opciones para el hiperparámetro k de SelectKBest 
grid.best_params_

{'kbest__k': 1}